In [1]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt

!mkdir media/
!gdown "https://drive.google.com/uc?id=1htMvTKT2PH3BPArrHrqwFqxdh000Zu9L" -O media/track1.mp4

Downloading...
From: https://drive.google.com/uc?id=1htMvTKT2PH3BPArrHrqwFqxdh000Zu9L
To: /content/media/track1.mp4
100% 472k/472k [00:00<00:00, 63.6MB/s]


### Optical Flow
Optical flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movement of object or camera. It is 2D vector field where each vector is a displacement vector showing the movement of points from first frame to second

![](https://devblogs.nvidia.com/wp-content/uploads/2019/02/Football-625x176.png)

It below image shows a ball moving in 5 consecutive frames. The arrow shows its displacement vector. 
![](https://docs.opencv.org/3.4/optical_flow_basic1.jpg)

Optical flow has many applications in areas like :

- Structure from Motion
- Video Compression
- Video Stabilization 
- Self localization ...  

Optical flow works on several assumptions:

1) The pixel intensities of an object do not change between consecutive frames.  
2) Neighbouring pixels have similar motion

#### Lucas–Kanade method
In computer vision, the Lucas–Kanade method is a widely used differential method for optical flow estimation developed by Bruce D. Lucas and Takeo Kanade. 

It assumes that the optical flow is essentially constant in a local neighbourhood around the pixel, and solves the over-determined basic optical flow equations for all the pixels in that neighbourhood, by the least squares criterion.  


Consider a pixel I(x,y,t) in the first frame. It moves by distance (dx,dy) in next frame taken after dt time. So since those pixels are the same and intensity does not change, we can say,

\begin{equation}
I(x,y,t) = I(x+dx,y+dy,t+dt)
\end{equation}
\begin{equation}
f_xu+f_yv+f_t=0
\end{equation}
      
\begin{equation}
fx=∂f/∂x;fy=∂f/∂y
\end{equation}
\begin{equation}
u=dx/dt;v=dy/dt
\end{equation}


In [2]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('media/track1.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [3]:
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

## How many persistent frame points to show
track_length = 10

def gen_points(roi, frame):
  '''
    A quick function to take 4X4 points inside the given ROI(Region of Interest)
  '''
  roi_points = []
  for i in range(4):
    for j in range(4):
      roi_points.append([ roi[0]+int(roi[2]*i/4) , roi[1]+int(roi[3]*j/4) ])
      cv2.circle(frame,(roi[0]+int(roi[2]*i/4) , roi[1]+int(roi[3]*j/4)),1,(0,255,0),-1)
  return roi_points

In [10]:
## Video saver object
out = cv2.VideoWriter(
    "media/track_out" + ".avi", cv2.VideoWriter_fourcc(*"MJPG"), 10.0,
    (426,240))

## Videocapture object
cap = cv2.VideoCapture("media/track1.mp4")
ret,frame = cap.read()

## Select the region to track
# roi = cv2.selectROI(frame)
roi = (131, 116, 30, 20)

old_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
roi_points = gen_points(roi, frame)

## Intial points to track in frame
p0 = np.array([roi_points],dtype=np.float32).reshape((-1,1,2))

## we would storing all the previous frame points in this
all_points = [p0]

while True:
    ret,frame = cap.read()
    if not ret:
      break
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)
    
    all_points.append(p1)
    for points in all_points:
        ## draw point for each frame
        for point in points:
            x,y = point.ravel()
            cv2.circle(frame,(x,y),1,(0,255,0),-1)


    old_gray = gray
    p0 = p1

    if len(all_points) == track_length:
        all_points.pop(0)

    out.write(frame)
        
out.release()    
cap.release()
cv2.destroyAllWindows()


In [11]:
!ffmpeg -hide_banner -loglevel warning -i media/track_out.avi -vf fps="fps=60" media/track_out.mp4 
from IPython.display import HTML
from base64 import b64encode
mp4 = open('media/track_out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

File 'media/track_out.mp4' already exists. Overwrite ? [y/N] y
